In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import keras

# 特征单词数，一个句子的总词语数
max_words = 25000
# 这些单词都属于max_words中的单词，输入维度
maxlen = 100
# 输出维度
embedding_dim = 64

class TextClassfyLSTM(keras.Model):

    def __init__(self, units,words):
        super(TextClassfyLSTM, self).__init__()

        self.units = units
        # 特征单词数，一个句子的总词语数
        max_words = words
        self.embedding = keras.layers.Embedding(max_words, embedding_dim, input_length=maxlen)

        self.lstm = keras.layers.Bidirectional(keras.layers.LSTM(self.units))

        self.dense = keras.layers.Dense(7, activation='softmax')

    def call(self, x, training=None, mask=None):
        x = self.embedding(x)
        x = self.lstm(x)
        x = self.dense(x)

        return x


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.metrics import classification_report
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

# 特征单词数
max_words = 0
# 这些单词都属于max_words中的单词，输入维度
maxlen = 100

batch_size = 20


def LSTM(X,Y,test_X,test_Y,max_words):
    model = TextClassfyLSTM(3, max_words)
    model.compile(optimizer="SGD",
                  loss="categorical_crossentropy",
                  metrics=['categorical_accuracy']
                  )
    history = model.fit(
        X,
        Y,
        batch_size=batch_size,
        epochs=50,
        validation_data=(test_X, test_Y)
    )
    # model.summary()
    score = model.evaluate(test_X, test_Y, verbose=0)
    print(score[1])
    return history,model
def tokenized(dataframe):
    # 1、对句子进行分词
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(dataframe)
    vocab = len(tokenizer.word_index)+1
    train_X = tokenizer.texts_to_sequences(dataframe)
    # 2、填充
    train_X = pad_sequences(train_X, maxlen=maxlen)
    return train_X, vocab

if __name__ == '__main__':
    # 数据预处理
    raw_data = pd.read_csv('/kaggle/input/feedback-prize-2021/train.csv')
    fresh_data = raw_data[['discourse_text', 'discourse_type']]
    #先拿部分进行实验
    train_data = fresh_data.iloc[0:15000, :]
    train_Y = pd.get_dummies(train_data.discourse_type)
    train_X, train_words = tokenized(train_data['discourse_text'])
    #测试集
    test_data = fresh_data.iloc[15000:, :]
    test_Y = pd.get_dummies(test_data.discourse_type)
    test_X, test_words = tokenized(test_data['discourse_text'])
    max_words = train_words+test_words
    #建立模型
    history, model = LSTM(train_X, train_Y, test_X, test_Y, max_words)
